![CrowdAI-Logo](https://github.com/crowdAI/crowdai/raw/master/app/assets/images/misc/crowdai-logo-smile.svg?sanitize=true)

# Mapping Challenge (Local Evaluation)

**Author** : [Sharada Mohanty](mailto:sharada.mohanty@epfl.ch)

This notebook walks you through the process of locally evaluating your submissions.   

In [26]:
import random
import json
import numpy as np
import argparse
import base64
import glob
import os

from pycocotools.coco import COCO
from cocoeval import COCOeval
# Note that, we use a slightly modified version of the official `COCOEval` class, 
# and it has been included in this repository for reference.

In [7]:
# Configuration Variables
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
padding = 50
SEGMENTATION_LENGTH = 10
MAX_NUMBER_OF_ANNOTATIONS = 10

IMAGES_DIR = "data/val/images"
ANNOTATION_PATH = "data/val/annotation.json"

To begin with, we will gather everything we discussed in the [Random Submission](https://github.com/crowdAI/mapping-challenge-starter-kit/blob/master/Random%20Submission.ipynb) notebook, and make it a single function that we can re-use in this notebook.

In [5]:
def generate_and_save_random_prediction(IMAGE_IDS):
    def bounding_box_from_points(points):
        """
            This function only supports the `poly` format.
        """
        points = np.array(points).flatten()
        even_locations = np.arange(points.shape[0]/2) * 2
        odd_locations = even_locations + 1
        X = np.take(points, even_locations.tolist())
        Y = np.take(points, odd_locations.tolist())
        bbox = [X.min(), Y.min(), X.max()-X.min(), Y.max()-Y.min()]
        bbox = [int(b) for b in bbox]
        return bbox

    def single_segmentation(image_width, image_height, number_of_points=10):
        points = []
        for k in range(number_of_points):
            # Choose a random x-coordinate
            random_x = int(random.randint(0, image_width))
            # Choose a random y-coordinate
            random_y = int(random.randint(0, image_height))
            #Flatly append them to the list of points
            points.append(random_x)
            points.append(random_y)
        return [points]

    def single_annotation(image_id, number_of_points=10):
        _result = {}
        _result["image_id"] = image_id
        _result["category_id"] = 100 # as 100 is the category_id of Building
        _result["score"] = np.random.rand() # a random score between 0 and 1

        _result["segmentation"] = single_segmentation(IMAGE_WIDTH, IMAGE_HEIGHT, number_of_points=number_of_points)
        _result["bbox"] = bounding_box_from_points(_result["segmentation"])
        return _result
    
    predictions = []
    for image_id in IMAGE_IDS:
        number_of_annotations = random.randint(0, MAX_NUMBER_OF_ANNOTATIONS)
        for _idx in range(number_of_annotations):
            _annotation = single_annotation(image_id)
            predictions.append(_annotation)
    
    import json
    fp = open("predictions.json", "w")
    fp.write(json.dumps(predictions))
    fp.close()    

We will focus on the validation set, and first generate a random prediction for the validation set.   

We will collect the image_ids this time by reading the actual list of files, and then we use the function we just defined to create a random `predictions.json` file.

In [11]:
image_paths = glob.glob(os.path.join(IMAGES_DIR, "*.jpg"))
IMAGE_IDS = [int(os.path.basename(x).replace(".jpg", "")) for x in image_paths]

In [12]:
generate_and_save_random_prediction(IMAGE_IDS)

# Preparing for evaluation

Before we can evaluate a submission, we will need the ground truth annotations, which we can load by : 

In [14]:
ground_truth_annotations = COCO(ANNOTATION_PATH)

loading annotations into memory...
Done (t=5.91s)
creating index...
index created!


..then we will need to actually open the `predictions.json` file.

In [16]:
submission_file = json.loads(open("predictions.json").read())

and now we load the results from the predictions file using the `loadRes` function from the `cocoapi`.

In [17]:
results = ground_truth_annotations.loadRes(submission_file)

Loading and preparing results...
DONE (t=0.48s)
creating index...
index created!


# Initiate Evaluation

We initiate the evaluation by using the `COCOEval` class and instantiating an evaluation for segmentation between the `ground_truth_annotations` and the `results`

In [18]:
cocoEval = COCOeval(ground_truth_annotations, results, 'segm')

# Compute and Accumulate Metrics

This step might take a few minutes, so please be patient.

In [23]:
cocoEval.evaluate()
cocoEval.accumulate()

Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=117.34s).
Accumulating evaluation results...
DONE (t=16.73s).


# Summarise Metrics

In [25]:
average_precision = cocoEval._summarize(ap=1, iouThr=0.5, areaRng="all", maxDets=100)
average_recall = cocoEval._summarize(ap=0, iouThr=0.5, areaRng="all", maxDets=100)
print("Average Precision : {} || Average Recall : {}".format(average_precision, average_recall))

 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
Average Precision : 2.2962201880565412e-06 || Average Recall : 0.00012806482408550074
